<a href="https://colab.research.google.com/github/hanlintao/langchain/blob/main/PDF2txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai
%pip install langchain==0.0.263

In [ ]:
import os

from langchain.document_loaders import PyPDFLoader
from langchain import PromptTemplate
#from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.callbacks import get_openai_callback

os.environ['OPENAI_API_KEY'] = "****"  #改成自己的key

In [ ]:
def save_to_txt(original_text, translated_text):
    with open('translated_text.txt', 'w', encoding='utf-8') as file:
        for original, translated in zip(original_text, translated_text):
            file.write(f'原文：{original}\n')
            file.write(f'译文：{translated}\n\n')

def read_pdf(pdf):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    split_pdf_num = text_splitter.split_documents(pdf)
    print(f'你的PDF文件被切割成了 {len(split_pdf_num)} 份')

    template = "请将下面的内容翻译中文：{chunk}"

    prompt = PromptTemplate(
        input_variables=["chunk"],
        template=template,
    )

    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-16k")

    total_cost = 0  # 初始化总费用
    original_texts = []  # 存储所有原文的列表
    translated_texts = []  # 存储所有翻译结果的列表

    for i, split_pdf in enumerate(split_pdf_num):
        print(f'\n--- 第{i+1}页 ---')

        cleaned_text = split_pdf.page_content.replace('\n', '')

        print(cleaned_text)

        with get_openai_callback() as cb:
            chain = LLMChain(llm=llm, prompt=prompt)
            translated_text = chain.run(cleaned_text)
            print(f'\n译文为：{translated_text}')
            print(f"\n本次消耗: {cb.total_cost} 美元")
            total_cost += cb.total_cost  # 更新总费用
            original_texts.append(cleaned_text)  # 将原文添加到列表中
            translated_texts.append(translated_text)  # 将翻译结果添加到列表中

    print(f"\n所有页面翻译完毕。总费用为: {total_cost} 美元")  # 所有循环结束后，打印总费用

    # 在循环结束后，传递原文列表和译文列表给save_to_txt()函数
    save_to_txt(original_texts, translated_texts)

In [ ]:
%pip install pypdf

In [ ]:
loader = PyPDFLoader("demo.pdf")
pages = loader.load_and_split()
#read_pdf(pages[:3])
read_pdf(pages)